In [4]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
import toniutils as tu

In [5]:
!pip install keras

    100% |████████████████████████████████| 378kB 19.5MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
from keras.models import model_from_json

Using TensorFlow backend.


<h2>Load Models</h2>

In [7]:
import numpy as np
import io
import pickle
import boto3
from keras.models import model_from_json

s3_client = boto3.client('s3')
#result = client.get_object(Bucket='bme-bucket', Key='models/keras-model-1.json')
#model = model_from_json(result["Body"].read())

In [8]:
models_dict ={}
for model in ['model-' + str(i) for i in range(1, 19)]:
    #print('models/keras-' + model + '.json')
    model_json = s3_client.get_object(Bucket='bme-bucket', Key='models/keras-' + model + '.json')
    models_dict[model] = model_from_json(model_json['Body'].read())

Instructions for updating:
Colocations handled automatically by placer.


<h2>Load Data</h2>

In [9]:
# download without using disk
my_array_data2 = io.BytesIO()
s3_client.download_fileobj('bme-bucket', 'engineered_data/Y_minmaxscaled.pkl', my_array_data2)
my_array_data2.seek(0)
Y = pickle.load(my_array_data2)

my_array_data2 = io.BytesIO()
s3_client.download_fileobj('bme-bucket', 'engineered_data/X.pkl', my_array_data2)
my_array_data2.seek(0)
X = pickle.load(my_array_data2)

<h3>Format data</h3>

In [10]:
X_2nd_layer = models_dict['model-1'].predict(X)

In [11]:
for i in range(2, 19):
    X_2nd_layer = np.hstack( (X_2nd_layer, models_dict['model-' + str(i)].predict(X)) )

<h2>2nd Layer Models</h2>

In [12]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [13]:
ESTIMATORS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10,
                                       max_features=32,     # Out of 20000
                                       random_state=0),
    #"K-nn": KNeighborsRegressor(), # To Lazy
    "Linear regression": LinearRegression(),
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
    "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
    "MultiO/P GBR": MultiOutputRegressor(GradientBoostingRegressor(n_estimators=5)),
    "MultiO/P AdaB": MultiOutputRegressor(AdaBoostRegressor(n_estimators=5)),
    "Regressor/C GBR": RegressorChain(GradientBoostingRegressor(n_estimators=5)),
    "Regressor/C AdaB": RegressorChain(AdaBoostRegressor(n_estimators=5))
}

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_2nd_layer, Y, test_size = 0.2, random_state = 2)

In [ ]:
FITTED_ESTIMATORS = {}
y_mse = {}

for name, estimator in ESTIMATORS.items():
    print(name)
    FITTED_ESTIMATORS[name] = estimator.fit(X_train, y_train)  # fit() with instantiated object
    y_mse[name] = mse(y_test, estimator.predict(X_test))

Extra trees
Linear regression
Ridge
Lasso
ElasticNet
RandomForestRegressor


/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Decision Tree Regressor
MultiO/P GBR


In [16]:
y_mse

{'Extra trees': 0.02438057632120357,
 'Linear regression': 0.024302767378201112,
 'Ridge': 0.033893122425497385,
 'Lasso': 0.051749155627251645,
 'ElasticNet': 0.051749155627251645,
 'RandomForestRegressor': 0.03569329276853517,
 'Decision Tree Regressor': 0.03587829797216047,
 'MultiO/P GBR': 0.03970889759634548,
 'MultiO/P AdaB': 0.03145953407549013,
 'Regressor/C GBR': 0.04668904784707298,
 'Regressor/C AdaB': 0.04190745797755727}

<h2>Save Models</h2>

In [18]:
import boto3
import io
import pickle

s3_client = boto3.client('s3')

In [19]:
my_array_data = io.BytesIO()
pickle.dump(FITTED_ESTIMATORS, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, 'bme-bucket', 'models/experiment-0/2nd-layer-models/models_dict.pkl')

In [20]:
my_array_data = io.BytesIO()
pickle.dump(y_mse, my_array_data)
my_array_data.seek(0)
s3_client.upload_fileobj(my_array_data, 'bme-bucket', 'models/experiment-0/2nd-layer-models_mse.pkl')